# 7.1 Sequential 모델을 넘어서: 케라스의 함수형 API

### `Sequential` 모델

- `Sequential` 모델은 네트워크의 입력과 출력이 하나
- 많은 경우에 이 모델이 적절하지만 `Sequential` 모델로 구현하지 못하는 경우도 있음
    - 다중 입력 모델
    - 다중 출력 모델
    - 비순환 유향 그래프 같은 네트워크 구조들


<img src="./images/sequential_model.jpeg" alt="sequential_model" align="left" />

### 1. 다중 입력 모델

- 다양한 입력 소스에서 전달된 데이터를 각각 다른 종류의 신경망으로 처리하고 합치는 형태

- 예) 중고 의류의 시장 가격 예측
    - 입력 : 메타데이터(의류브랜드, 연도 등), 사용자가 제공한 텍스트 설명, 제품 사진
    - 신경망 : FCN, RNN, CNN
    - 출력 : 가격
    
    
<img src="./images/multi_input_model.png" alt="multi_input_model" align="left" />

### 2. 다중 출력 모델

- 입력 데이터에서 여러개의 target 속성을 예측하는 경우

- 예) 소설이나 짧은 글의 장르 및 시대 예측
    - 입력 : 소설 텍스트
    - 신경망 : 분류 모델, 회귀 모델
    - 출력 : 장르, 시대
    

<img src="./images/multi_output_model.png" alt="multi_output_model" align="left" />

### 3. 비순환 유향 그래프 형태의 모델

- 최근에 개발된 많은 신경망 구조는 비순환 유향 그래프 형태를 가짐

####  inception module을 사용한 Inception 계열의 네트워크

- 입력을 나란히 놓인 여러개의 CNN을 통과시킨 후 하나의 텐서로 합침


<img src="./images/inception_module.png" alt="inception_module" align="left" />

#### residual connection을 이용한 ResNet 계열의 네트워크

- 하위 층의 출력 텐서를 상위 층의 출력 텐서에 더함
- 하위층에서 학습된 정보가 데이터 처리 과정에서 손실되는 것을 방지


<img src="./images/residual_connection.png" alt="residual_connection" align="left" />

## 7.1.1 함수형 API 소개

### 함수형 API

- 함수형 API(functional API)는 함수처럼 층을 사용하며 직접 텐서들의 입출력을 다룸

- 케라스의 `Model` 클래스를 통해 구현

#### `Model` 클래스

- 입력 텐서(`input_tensor`)와 출력 텐서(`output_tensor`)를 받음
- 입력 텐서에서 출력 텐서로 가는데 필요한 모든 층을 추출
- 추출한 모든 층을 모아서 그래프 데이터 구조인 `Model` 객체를 생성


- 관련되지 않은 입력과 출력으로 모델을 만들면 `RuntimeError` 발생


### Sequential 모델과 함수형 API의 비교 예제

In [1]:
# Sequential 모델

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, optimizers

seq_model = Sequential()

seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

seq_model.summary()

# compile()이후 단계는 동일함

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [2]:
# Functional API

from tensorflow.keras import Model, Input
from tensorflow.keras import layers

input_tensor = Input(shape=(64,))

x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)

output_tensor = layers.Dense(10, activation='softmax')(x)

func_model = Model(input_tensor, output_tensor)

func_model.summary()

# compile()이후 단계는 동일함

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


## 7.1.2 다중 입력 모델

### 다중 입력 모델

- Functional API를 사용하면 다중 입력 모델을 만들 수 있음
- 서로 다른 입력을 합치기 위해 텐서를 연결하는 층을 사용함
    - `keras.layers.Add()`, `keras.layers.Concatenate()` 등
    - 위의 2가지 외에도 여러가지 함수들이 있음(케라스의 Merge layer 참고)
    
### 다중 입력 모델 예제 : 질문-응답 모델

- 입력 : 자연어 질문, 답변에 필요한 정보가 담긴 텍스트(뉴스기사 등)
- 신경망 : LSTM, LSTM
- 출력 : 단어로 응답

In [3]:
# Functional API로 질문-응답 모델 구현

from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Embedding, LSTM, Dense, Concatenate
from tensorflow.keras.optimizers import RMSprop

text_voca_size = 10000
question_voca_size = 10000

answer_voca_size = 500


# 1. 정보 텍스트

# shape=(None,) : 입력 텍스트의 길이가 정해지지 않음
text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = Embedding(text_voca_size, 64)(text_input)
encoded_text = LSTM(32)(embedded_text)

# 2. 자연어 질문

question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = Embedding(question_voca_size, 32)(question_input)
encoded_question = LSTM(16)(embedded_question)

# 1과 2의 출력을 하나로 합침

# axis 매개변수는 -1이 기본값이므로 생략해도 됨
concatenated = Concatenate(axis=-1)([encoded_text, encoded_question])

answer = Dense(answer_voca_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None, None)]       0                                            
__________________________________________________________________________________________________
question (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 64)     640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 32)     320000      question[0][0]                   
____________________________________________________________________________________________

In [4]:
# 구현을 위해 난수로 데이터 생성

import numpy as np
from tensorflow.keras.utils import to_categorical

num_samples = 1000
max_length = 100

# 데이터 생성
text = np.random.randint(1, text_voca_size, size=(num_samples, max_length))
question = np.random.randint(1, question_voca_size, size=(num_samples, max_length))
answers = np.random.randint(0, answer_voca_size, size=num_samples)


# ont-hot encoding
answers = to_categorical(answers)

print(text.shape)
print(question.shape)
print(answers.shape)

(1000, 100)
(1000, 100)
(1000, 500)


In [5]:
# 학습 시키기

model.compile(optimizer=RMSprop(),
              loss='categorical_crossentropy',
              metrics=['acc'])

# 방법 1. 리스트로 주입
model.fit([text, question], answers,
          epochs=10, batch_size=128)

# 방법 2. Input 클래스의 name 매개변수로 전달한 이름으로 딕셔너리 만들어서 전달
model.fit({'text':text, 'question':question}, answers,
          epochs=10, batch_size=128)

Train on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 4s 4ms/sample - loss: 6.2146 - acc: 0.0010
Epoch 2/10
1000/1000 [==============================] - 0s 127us/sample - loss: 6.1943 - acc: 0.0200
Epoch 3/10
1000/1000 [==============================] - 0s 115us/sample - loss: 6.1191 - acc: 0.0060
Epoch 4/10
1000/1000 [==============================] - 0s 112us/sample - loss: 6.0409 - acc: 0.0060
Epoch 5/10
1000/1000 [==============================] - 0s 112us/sample - loss: 5.9839 - acc: 0.0070
Epoch 6/10
1000/1000 [==============================] - 0s 116us/sample - loss: 5.9122 - acc: 0.0110
Epoch 7/10
1000/1000 [==============================] - 0s 115us/sample - loss: 5.8339 - acc: 0.0160
Epoch 8/10
1000/1000 [==============================] - 0s 112us/sample - loss: 5.7333 - acc: 0.0150
Epoch 9/10
1000/1000 [==============================] - 0s 117us/sample - loss: 5.6616 - acc: 0.0280
Epoch 10/10
1000/1000 [==============================] - 0s 114us/sampl

## 7.1.3 다중 출력 모델

### 다중 출력 모델

- Functional API를 사용하면 다중 출력 모델을 만들 수 있음
- 서로 다른 출력별로 각각 서로 다른 손실함수를 지정하고 손실값을 하나로 합쳐서 경사하강법을 적용
    - 경사 하강법은 하나의 스칼라 값을 최소화 하므로 손실 값을 하나로 합침
    

#### 손실 값의 불균형
- 서로 다른 손실 값이 불균형하면 개별 손실 값이 큰 작업에 치우쳐서 학습되므로 다른 작업들이 손해를 입음
- 이를 해결하기 위해 각 손실 값별로 전체 손실에 기여하는 수준(가중치)을 적용할 수 있음
    - `compile()`의 `loss_weights` 매개변수에 리스트로 전달
    
    
- 서로 다른 2개의 loss를 가지는 경우의 예
    - loss1
        - 일반적으로 3~5 값을 가짐
        - 가중치를 0.25로 줌
        - loss1*0.25 = 0.75~1.25
    - loss2
        - 일반적으로 0.1정도의 값을 가짐
        - 가중치를 10으로 줌
        - loss2*10 = 1
    
    
### 다중 출력 모델 예제 : sns의 포스트를 통한 나이, 성별, 소득수준 예측 문제

- 입력 : sns의 포스트(텍스트)
- 신경망 : 1D CNN
- 출력 : 회귀, 다중분류, 이진분류

In [7]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Embedding, Dense, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.optimizers import RMSprop

voca_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')

embedding_posts = Embedding(voca_size, 256)(posts_input)

x = Conv1D(128, 5, activation='relu')(embedding_posts)
x = MaxPooling1D(5)(x)

x = Conv1D(256, 5, activation='relu')(x)
x = Conv1D(256, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)

x = Conv1D(256, 5, activation='relu')(x)
x = Conv1D(256, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)

x = Dense(128, activation='relu')(x)

age_prediction = Dense(1, name='age')(x)
gender_prediction = Dense(2, name='gender', activation='sigmoid')(x)
income_prediction = Dense(num_income_groups, name='income', activation='softmax')(x)

model = Model(posts_input, [age_prediction, gender_prediction, income_prediction])

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 256)    12800000    posts[0][0]                      
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, None, 128)    163968      embedding_3[0][0]                
__________________________________________________________________________________________________
max_pooling1d_2 (MaxPooling1D)  (None, None, 128)    0           conv1d_5[0][0]                   
____________________________________________________________________________________________

In [ ]:
# 학습시키기

# train, target 데이터가 만들어져있다고 가정

# 각 loss가 아래와 같이 나온다고 가정
# mse loss : 3~5,
# binary crossentropy loss : 0.1
# categorical crossentropy loss : 1

# 방법 1 : 순서대로 리스트로 넣기

model.compile(optimizer=RMSprop,
              loss=['mse', 'binary_crossentropy', 'categorical_crossentropy'],
              loss_weights=[0.25, 10., 1.])

model.fit(posts, [age_targets, gender_targets, income_targets],
          epochs=10, batch_size=64)


# 방법 2 : 이름 지정해서 넣기

model.compile(optimizer=RMSprop,
              loss={'age':'mse',
                    'gender':'binary_crossentropy',
                    'income':'categorical_crossentropy'},
              loss_weights={'age':0.25,
                            'gender':10.,
                            'income':1.})

model.fit(posts, {'age': age_targets,
                  'gender': gender_targets,
                  'income': income_targets},
          epochs=10, batch_size=64)